# #05 Simple frame structure
<i>Creates a simple frame structure based on a 2d grid and storey height inputs</i>
***

Geometry definition:

In [1]:
''' Inputs '''
# Define column spacings and storey heights in m
column_spacing_x = [5.000, 6.000, 6.000, 5.000]
column_spacing_y = [3.000, 4.500, 3.000]
storey_heights = [4.500, 3.000, 3.000, 3.000]

# Start coordinates. You can run this script multiple times in the same model with different offsets to build more complex geometries
origin_x = 0.0
origin_y = 0.0

Connect to LUSAS Modeller and create a new model

In [ ]:
from shared.LPI import *
lusas = get_lusas_modeller()
if lusas.existsDatabase() and lusas.db().isModified():
    raise Exception("This script will create a new model. Please save or close the current model and try again")

# Create a new model
lusas.newProject("Structural", "Simple_Frame_Model.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits(lusas.getUnitSet("N,m,t,s,C"))


Calculate the x and y coordinates of the frame grid

In [ ]:
import numpy as np
column_positions_x = np.add.accumulate(column_spacing_x) + origin_x
column_positions_x = np.insert(column_positions_x, 0, origin_x)

column_positions_y = np.add.accumulate(column_spacing_y) + origin_y
column_positions_y = np.insert(column_positions_y, 0, origin_y)

Generate the model lines. We'll let Modeller merge the points as we create the lines

In [ ]:
# Reference to the geometryData object
geom_data = lusas.geometryData()

z_bot = 0.0 # Vertical coordinate of the bottom of the storey
z_top = 0.0 # Vertical coordinate of the top of the storey
for height in storey_heights:
    z_top = z_bot + height

    # Columns
    for i, x in enumerate(column_positions_x):
        for j, y in enumerate(column_positions_y):
            geom_data.setAllDefaults()          # Reset the inputs
            geom_data.addCoords(x, y, z_bot)    # Set the coordinates of the first point
            geom_data.addCoords(x, y, z_top)    # Set the coordinates of the second point
            geom_data.setLowerOrderGeometryType("coordinates")
            geom_data.setCreateMethod("straight")
            db.createLine(geom_data)            # Create the column line

    # Beams in y
    for i, x in enumerate(column_positions_x):
        for j, y in enumerate(column_positions_y):
            if j < len(column_positions_y)-1:
                geom_data.setAllDefaults()
                geom_data.addCoords(x, y, z_top)
                geom_data.addCoords(x, column_positions_y[j+1], z_top)
                geom_data.setLowerOrderGeometryType("coordinates")
                geom_data.setCreateMethod("straight")
                db.createLine(geom_data)

    # Beams in x
    for i, x in enumerate(column_positions_x):
        for j, y in enumerate(column_positions_y):
            if i < len(column_positions_x)-1:
                geom_data.setAllDefaults()
                geom_data.addCoords(x, y, z_top)
                geom_data.addCoords(column_positions_x[i+1], y, z_top)
                geom_data.setLowerOrderGeometryType("coordinates")
                geom_data.setCreateMethod("straight")
                db.createLine(geom_data)

    z_bot = z_top # next storey

Change from plan view to top-front-side view (isometric view)

In [10]:
lusas.view().setIsometric()